# Feature Store taxi example notebook

This notebook illustrates the use of Feature Store to create a model that predicts NYC Yellow Taxi fares. It includes these steps:

- Compute and write features.
- Train a model using these features to predict fares.
- Evaluate that model on a new batch of data using existing features, saved to Feature Store.

## Requirements
- Databricks Runtime for Machine Learning 8.3 or above.

<img src="https://docs.databricks.com/_static/images/machine-learning/feature-store/taxi_example_flow_v3.png"/>

## Compute features

#### Load the raw data used to compute features

Load the `nyc-taxi-tiny` dataset.  This was generated from the full [NYC Taxi Data](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page) which can be found at `dbfs:/databricks-datasets/nyctaxi` by applying the following transformations:

1. Apply a UDF to convert latitude and longitude coordinates into ZIP codes, and add a ZIP code column to the DataFrame.
1. Subsample the dataset into a smaller dataset based on a date range query using the `.sample()` method of the Spark `DataFrame` API.
1. Rename certain columns and drop unnecessary columns.

If you want to create this dataset from the raw data yourself, follow these steps:
1. Run the Feature Store taxi example dataset notebook ([AWS](https://docs.databricks.com/_static/notebooks/machine-learning/feature-store-taxi-example-dataset.html)|[Azure](https://docs.microsoft.com/azure/databricks/_static/notebooks/machine-learning/feature-store-taxi-example-dataset.html)|[GCP](https://docs.gcp.databricks.com/_static/notebooks/machine-learning/feature-store-taxi-example-dataset.html)) to generate the Delta table.
1. In this notebook, replace the following `spark.read.format("delta").load("/databricks-datasets/nyctaxi-with-zipcodes/subsampled")` with: `spark.read.table("feature_store_taxi_example.nyc_yellow_taxi_with_zips")`

In [0]:
raw_data = spark.read.format("delta").load("/databricks-datasets/nyctaxi-with-zipcodes/subsampled")
display(raw_data)

tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,fare_amount,pickup_zip,dropoff_zip
2016-02-14T16:52:13.000+0000,2016-02-14T17:16:04.000+0000,4.94,19.0,10282,10171
2016-02-04T18:44:19.000+0000,2016-02-04T18:46:00.000+0000,0.28,3.5,10110,10110
2016-02-17T17:13:57.000+0000,2016-02-17T17:17:55.000+0000,0.7,5.0,10103,10023
2016-02-18T10:36:07.000+0000,2016-02-18T10:41:45.000+0000,0.8,6.0,10022,10017
2016-02-22T14:14:41.000+0000,2016-02-22T14:31:52.000+0000,4.51,17.0,10110,10282
2016-02-05T06:45:02.000+0000,2016-02-05T06:50:26.000+0000,1.8,7.0,10009,10065
2016-02-15T15:03:28.000+0000,2016-02-15T15:18:45.000+0000,2.58,12.0,10153,10199
2016-02-25T19:09:26.000+0000,2016-02-25T19:24:50.000+0000,1.4,11.0,10112,10069
2016-02-13T16:28:18.000+0000,2016-02-13T16:36:36.000+0000,1.21,7.5,10023,10153
2016-02-14T00:03:48.000+0000,2016-02-14T00:10:24.000+0000,0.6,6.0,10012,10003


In [0]:
display(raw_data)

tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,fare_amount,pickup_zip,dropoff_zip
2016-02-14T16:52:13.000+0000,2016-02-14T17:16:04.000+0000,4.94,19.0,10282,10171
2016-02-04T18:44:19.000+0000,2016-02-04T18:46:00.000+0000,0.28,3.5,10110,10110
2016-02-17T17:13:57.000+0000,2016-02-17T17:17:55.000+0000,0.7,5.0,10103,10023
2016-02-18T10:36:07.000+0000,2016-02-18T10:41:45.000+0000,0.8,6.0,10022,10017
2016-02-22T14:14:41.000+0000,2016-02-22T14:31:52.000+0000,4.51,17.0,10110,10282
2016-02-05T06:45:02.000+0000,2016-02-05T06:50:26.000+0000,1.8,7.0,10009,10065
2016-02-15T15:03:28.000+0000,2016-02-15T15:18:45.000+0000,2.58,12.0,10153,10199
2016-02-25T19:09:26.000+0000,2016-02-25T19:24:50.000+0000,1.4,11.0,10112,10069
2016-02-13T16:28:18.000+0000,2016-02-13T16:36:36.000+0000,1.21,7.5,10023,10153
2016-02-14T00:03:48.000+0000,2016-02-14T00:10:24.000+0000,0.6,6.0,10012,10003


From the taxi fares transactional data, we will compute two groups of features based on trip pickup and drop off zip codes.

#### Pickup features
1. Count of trips (time window = 1 hour, sliding window = 15 minutes)
1. Mean fare amount (time window = 1 hour, sliding window = 15 minutes)

#### Drop off features
1. Count of trips (time window = 30 minutes)
1. Does trip end on the weekend (custom feature using python code)

<img src="https://docs.databricks.com/_static/images/machine-learning/feature-store/taxi_example_computation_v5.png"/>

### Helper functions

In [0]:
from databricks import feature_store
from pyspark.sql.functions import *
from pyspark.sql.types import FloatType, IntegerType, StringType
from pytz import timezone


@udf(returnType=IntegerType())
def is_weekend(dt):
    tz = "America/New_York"
    return int(dt.astimezone(timezone(tz)).weekday() >= 5)  # 5 = Saturday, 6 = Sunday
  
@udf(returnType=StringType())  
def partition_id(dt):
    # datetime -> "YYYY-MM"
    return f"{dt.year:04d}-{dt.month:02d}"


def filter_df_by_ts(df, ts_column, start_date, end_date):
    if ts_column and start_date:
        df = df.filter(col(ts_column) >= start_date)
    if ts_column and end_date:
        df = df.filter(col(ts_column) < end_date)
    return df


### Data scientist's custom code to compute features

In [0]:
def pickup_features_fn(df, ts_column, start_date, end_date):
    """
    Computes the pickup_features feature group.
    To restrict features to a time range, pass in ts_column, start_date, and/or end_date as kwargs.
    """
    df = filter_df_by_ts(
        df, ts_column, start_date, end_date
    )
    pickupzip_features = (
        df.groupBy(
            "pickup_zip", window("tpep_pickup_datetime", "1 hour", "15 minutes")
        )  # 1 hour window, sliding every 15 minutes
        .agg(
            mean("fare_amount").alias("mean_fare_window_1h_pickup_zip"),
            count("*").alias("count_trips_window_1h_pickup_zip"),
        )
        .select(
            col("pickup_zip").alias("zip"),
            unix_timestamp(col("window.end")).alias("ts").cast(IntegerType()),
            partition_id(to_timestamp(col("window.end"))).alias("yyyy_mm"),
            col("mean_fare_window_1h_pickup_zip").cast(FloatType()),
            col("count_trips_window_1h_pickup_zip").cast(IntegerType()),
        )
    )
    return pickupzip_features
  
def dropoff_features_fn(df, ts_column, start_date, end_date):
    """
    Computes the dropoff_features feature group.
    To restrict features to a time range, pass in ts_column, start_date, and/or end_date as kwargs.
    """
    df = filter_df_by_ts(
        df,  ts_column, start_date, end_date
    )
    dropoffzip_features = (
        df.groupBy("dropoff_zip", window("tpep_dropoff_datetime", "30 minute"))
        .agg(count("*").alias("count_trips_window_30m_dropoff_zip"))
        .select(
            col("dropoff_zip").alias("zip"),
            unix_timestamp(col("window.end")).alias("ts").cast(IntegerType()),
            partition_id(to_timestamp(col("window.end"))).alias("yyyy_mm"),
            col("count_trips_window_30m_dropoff_zip").cast(IntegerType()),
            is_weekend(col("window.end")).alias("dropoff_is_weekend"),
        )
    )
    return dropoffzip_features  

In [0]:
from datetime import datetime

pickup_features = pickup_features_fn(
    raw_data, ts_column="tpep_pickup_datetime", start_date=datetime(2016, 1, 1), end_date=datetime(2016, 1, 31)
)
dropoff_features = dropoff_features_fn(
    raw_data, ts_column="tpep_dropoff_datetime", start_date=datetime(2016, 1, 1), end_date=datetime(2016, 1, 31)
)

In [0]:
display(pickup_features)

zip,ts,yyyy_mm,mean_fare_window_1h_pickup_zip,count_trips_window_1h_pickup_zip
10119,1453978800,2016-01,10.5,2
10199,1453138200,2016-01,6.0,1
10022,1453964400,2016-01,10.5,1
10282,1452094200,2016-01,6.5,1
10011,1452780000,2016-01,5.0,1
10110,1453070700,2016-01,8.0,1
10007,1454035500,2016-01,26.5,1
10011,1454032800,2016-01,3.5,1
11201,1452903300,2016-01,7.5,1
11104,1452996000,2016-01,5.5,1


In [0]:
pickup_features.createOrReplaceTempView("pickup_features")
display(spark.sql("select * from pickup_features where zip = 10119 order by ts"))

zip,ts,yyyy_mm,mean_fare_window_1h_pickup_zip,count_trips_window_1h_pickup_zip
10119,1451618100,2016-01,7.5,1
10119,1451619000,2016-01,7.5,1
10119,1451619900,2016-01,7.5,1
10119,1451620800,2016-01,7.5,1
10119,1451673000,2016-01,10.5,1
10119,1451673900,2016-01,10.5,1
10119,1451674800,2016-01,10.5,1
10119,1451675700,2016-01,10.5,1
10119,1451677500,2016-01,6.0,1
10119,1451678400,2016-01,6.0,1


### Use Feature Store library to create new feature tables

First, create the database where the feature tables will be stored.

In [0]:
%sql 
CREATE DATABASE IF NOT EXISTS feature_store_taxi_example;

Next, create an instance of the Feature Store client.

In [0]:
fs = feature_store.FeatureStoreClient()

Use the `create_feature_table` API to define schema and unique ID keys. If the optional `features_df` argument is passed, the API also writes the data to Feature Store.

In [0]:
sqlContext.setConf("spark.sql.shuffle.partitions", "5")

fs.create_feature_table(
    name="feature_store_taxi_example.trip_pickup_features",
    keys=["zip", "ts"],
    features_df=pickup_features,
    partition_columns="yyyy_mm",
    description="Taxi Fares. Pickup Features",
)
fs.create_feature_table(
    name="feature_store_taxi_example.trip_dropoff_features",
    keys=["zip", "ts"],
    features_df=dropoff_features,
    partition_columns="yyyy_mm",
    description="Taxi Fares. Dropoff Features",
)

2021/08/18 12:21:11 INFO databricks.feature_store.utils.spark_listener: Actively listening to data sources.
2021/08/18 12:21:28 INFO databricks.feature_store.utils.spark_listener: Actively listening to data sources.
Out[12]: <FeatureTable: >

In [0]:
test = fs.read_table(
  name='feature_store_taxi_example.trip_pickup_features',
  as_of_delta_timestamp = str('2021-08-18 12:21:19')
)

In [0]:
display(test)

zip,ts,yyyy_mm,mean_fare_window_1h_pickup_zip,count_trips_window_1h_pickup_zip
10023,1452818700,2016-01,10.0,1
10023,1452819600,2016-01,10.0,1
10003,1453967100,2016-01,6.75,2
10003,1453968000,2016-01,6.75,2
10153,1452114000,2016-01,12.0,3
10153,1452114900,2016-01,12.0,3
10153,1452797100,2016-01,4.5,1
10153,1452798000,2016-01,4.5,1
10022,1451664000,2016-01,7.0,1
10103,1452379500,2016-01,14.5,1


In [0]:
%sql 
DESCRIBE HISTORY feature_store_taxi_example.trip_pickup_features

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
2,2021-08-18T12:56:46.000+0000,7278444638023685,saurabh.verma2@adidas.com,MERGE,"Map(predicate -> ((result_table.`zip` = updates_table.`zip`) AND (result_table.`ts` = updates_table.`ts`)), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(3790025709814458),0818-115721-kegs425,1,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, executionTimeMs -> 4243, numTargetRowsInserted -> 32243, scanTimeMs -> 2019, numTargetRowsUpdated -> 0, numOutputRows -> 32243, numTargetChangeFilesAdded -> 0, numSourceRows -> 32243, numTargetFilesRemoved -> 0, rewriteTimeMs -> 2171)",null
1,2021-08-18T12:21:19.000+0000,7278444638023685,saurabh.verma2@adidas.com,CREATE OR REPLACE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [""yyyy_mm""], properties -> {})",null,List(3790025709814458),0818-115721-kegs425,0,WriteSerializable,false,"Map(numFiles -> 1, numOutputBytes -> 136857, numOutputRows -> 32313)",null
0,2021-08-18T12:21:01.000+0000,7278444638023685,saurabh.verma2@adidas.com,CREATE TABLE AS SELECT,"Map(isManaged -> true, description -> null, partitionBy -> [""yyyy_mm""], properties -> {})",null,List(3790025709814458),0818-115721-kegs425,null,SnapshotIsolation,true,"Map(numFiles -> 0, numOutputBytes -> 0, numOutputRows -> 0)",null


## Update features

Use the `write_table` function to update the feature table values.

<img src="https://docs.databricks.com/_static/images/machine-learning/feature-store/taxi_example_compute_and_write.png"/>

In [0]:
display(raw_data)

tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,fare_amount,pickup_zip,dropoff_zip
2016-02-14T16:52:13.000+0000,2016-02-14T17:16:04.000+0000,4.94,19.0,10282,10171
2016-02-04T18:44:19.000+0000,2016-02-04T18:46:00.000+0000,0.28,3.5,10110,10110
2016-02-17T17:13:57.000+0000,2016-02-17T17:17:55.000+0000,0.7,5.0,10103,10023
2016-02-18T10:36:07.000+0000,2016-02-18T10:41:45.000+0000,0.8,6.0,10022,10017
2016-02-22T14:14:41.000+0000,2016-02-22T14:31:52.000+0000,4.51,17.0,10110,10282
2016-02-05T06:45:02.000+0000,2016-02-05T06:50:26.000+0000,1.8,7.0,10009,10065
2016-02-15T15:03:28.000+0000,2016-02-15T15:18:45.000+0000,2.58,12.0,10153,10199
2016-02-25T19:09:26.000+0000,2016-02-25T19:24:50.000+0000,1.4,11.0,10112,10069
2016-02-13T16:28:18.000+0000,2016-02-13T16:36:36.000+0000,1.21,7.5,10023,10153
2016-02-14T00:03:48.000+0000,2016-02-14T00:10:24.000+0000,0.6,6.0,10012,10003


In [0]:
#Count before insertion of next set of features
pickup_features.count()

Out[71]: 32313

In [0]:
# Compute the pickup_features feature group.
pickup_features_df = pickup_features_fn(
  df=raw_data,
  ts_column="tpep_pickup_datetime",
  start_date=datetime(2016, 2, 1),
  end_date=datetime(2016, 2, 29),
)

# Write the pickup features DataFrame to the feature store table
fs.write_table(
  name="feature_store_taxi_example.trip_pickup_features",
  df=pickup_features_df,
  mode="merge",
)

# Compute the dropoff_features feature group.
dropoff_features_df = dropoff_features_fn(
  df=raw_data,
  ts_column="tpep_dropoff_datetime",
  start_date=datetime(2016, 2, 1),
  end_date=datetime(2016, 2, 29),
)

# Write the dropoff features DataFrame to the feature store table
fs.write_table(
  name="feature_store_taxi_example.trip_dropoff_features",
  df=dropoff_features_df,
  mode="merge",
)

2021/08/18 12:56:38 INFO databricks.feature_store.utils.spark_listener: Actively listening to data sources.
2021/08/18 12:56:50 INFO databricks.feature_store.utils.spark_listener: Actively listening to data sources.

In [0]:
test2 = fs.read_table(
  name='feature_store_taxi_example.trip_pickup_features',
#   as_of_delta_timestamp = str('2021-08-18 12:21:19')
)

In [0]:
#Count after insertion of next set of features
test2.createOrReplaceTempView("test2")
display(spark.sql("select count(*) from test2"))

count(1)
64556


When writing, both `merge` and `overwrite` modes are supported.

    fs.write_table(
      name="feature_store_taxi_example.trip_pickup_features",
      df=pickup_features_df,
      mode="overwrite",
    )
    
Data can also be streamed into Feature Store by passing a dataframe where `df.isStreaming` is set to `True`:

    fs.write_table(
      name="streaming_example.streaming_features",
      df=streaming_df,
      mode="merge",
    )
    
You can schedule a notebook to periodically update features using Databricks Jobs ([AWS](https://docs.databricks.com/jobs.html)|[Azure](https://docs.microsoft.com/azure/databricks/jobs)|[GCP](https://docs.gcp.databricks.com/jobs.html)).

Analysts can interact with Feature Store using SQL, for example:

In [0]:
%sql
SELECT SUM(count_trips_window_30m_dropoff_zip) AS num_rides,
       dropoff_is_weekend
FROM   feature_store_taxi_example.trip_dropoff_features
WHERE  dropoff_is_weekend IS NOT NULL
GROUP  BY dropoff_is_weekend;

num_rides,dropoff_is_weekend
15564,0
5670,1


## Feature Search and Discovery

You can now discover your feature tables in the <a href="#feature-store/" target="_blank">Feature Store UI</a>.

<img src="https://docs.databricks.com/_static/images/machine-learning/feature-store/taxi_example_flow_v3.png"/>

Search by "trip_pickup_features" or "trip_dropoff_features" to view details such as table schema, metadata, data sources, producers, and online stores. 

You can also edit the description for the feature table, or configure permissions for a feature table using the dropdown icon next to the feature table name. 

Check the [Use the Feature Store UI
](https://docs.databricks.com/applications/machine-learning/feature-store.html#use-the-feature-store-ui) documentation for more details.

## Train a model

This section illustrates how to train a model using the pickup and dropoff features stored in Feature Store. It trains a LightGBM model to predict taxi fare.

### Helper functions

In [0]:
from pyspark.sql import *
from pyspark.sql.functions import current_timestamp
from pyspark.sql.types import IntegerType
import math
from datetime import timedelta
import mlflow.pyfunc


def rounded_unix_timestamp(dt, num_minutes=15):
    """
    Ceilings datetime dt to interval num_minutes, then returns the unix timestamp.
    """
    nsecs = dt.minute * 60 + dt.second + dt.microsecond * 1e-6
    delta = math.ceil(nsecs / (60 * num_minutes)) * (60 * num_minutes) - nsecs
    return int((dt + timedelta(seconds=delta)).timestamp())


rounded_unix_timestamp_udf = udf(rounded_unix_timestamp, IntegerType())


def rounded_taxi_data(taxi_data_df):
    # Round the taxi data timestamp to 15 and 30 minute intervals so we can join with the pickup and dropoff features
    # respectively.
    taxi_data_df = (
        taxi_data_df.withColumn(
            "rounded_pickup_datetime",
            rounded_unix_timestamp_udf(taxi_data_df["tpep_pickup_datetime"], lit(15)),
        )
        .withColumn(
            "rounded_dropoff_datetime",
            rounded_unix_timestamp_udf(taxi_data_df["tpep_dropoff_datetime"], lit(30)),
        )
        .drop("tpep_pickup_datetime")
        .drop("tpep_dropoff_datetime")
    )
    taxi_data_df.createOrReplaceTempView("taxi_data")
    return taxi_data_df
  
def get_latest_model_version(model_name):
  latest_version = 1
  mlflow_client = MlflowClient()
  for mv in mlflow_client.search_model_versions(f"name='{model_name}'"):
    version_int = int(mv.version)
    if version_int > latest_version:
      latest_version = version_int
  return latest_version

### Read taxi data for training

In [0]:
taxi_data = rounded_taxi_data(raw_data)

### Understanding how a training dataset is created

In order to train a model, you need to create a training dataset that is used to train the model.  The training dataset is comprised of:

1. Raw input data
1. Features from the feature store

The raw input data is needed because it contains:

1. Primary keys used to join with features.
1. Raw features like `trip_distance` that are not in the feature store.
1. Prediction targets like `fare` that are required for model training.

Here's a visual overview that shows the raw input data being combined with the features in the Feature Store to produce the training dataset:

<img src="https://docs.databricks.com/_static/images/machine-learning/feature-store/taxi_example_feature_lookup.png"/>

These concepts are described further in the Creating a Training Dataset documentation ([AWS](https://docs.databricks.com/applications/machine-learning/feature-store.html#create-a-training-dataset)|[Azure](https://docs.microsoft.com/en-us/azure/databricks/applications/machine-learning/feature-store#create-a-training-dataset)|[GCP](https://docs.gcp.databricks.com/applications/machine-learning/feature-store.html#create-a-training-dataset)).

The next cell loads features from Feature Store for model training by creating a `FeatureLookup` for each needed feature.

In [0]:
from databricks.feature_store import FeatureLookup
import mlflow

pickup_features_table = "feature_store_taxi_example.trip_pickup_features"
dropoff_features_table = "feature_store_taxi_example.trip_dropoff_features"

pickup_feature_lookups = [
    FeatureLookup( 
      table_name = pickup_features_table,
      feature_name = "mean_fare_window_1h_pickup_zip",
      lookup_key = ["pickup_zip", "rounded_pickup_datetime"],
    ),
    FeatureLookup( 
      table_name = pickup_features_table,
      feature_name = "count_trips_window_1h_pickup_zip",
      lookup_key = ["pickup_zip", "rounded_pickup_datetime"],
    ),
]

dropoff_feature_lookups = [
    FeatureLookup( 
      table_name = dropoff_features_table,
      feature_name = "count_trips_window_30m_dropoff_zip",
      lookup_key = ["dropoff_zip", "rounded_dropoff_datetime"],
    ),
    FeatureLookup( 
      table_name = dropoff_features_table,
      feature_name = "dropoff_is_weekend",
      lookup_key = ["dropoff_zip", "rounded_dropoff_datetime"],
    ),
]

### Create a Training Dataset

When `fs.create_training_set(..)` is invoked below, the following steps will happen:

1. A `TrainingSet` object will be created, which will select specific features from Feature Store to use in training your model. Each feature is specified by the `FeatureLookup`'s created above. 

1. Features are joined with the raw input data according to each `FeatureLookup`'s `lookup_key`.

The `TrainingSet` is then transformed into a DataFrame to train on. This DataFrame includes the columns of taxi_data, as well as the features specified in the `FeatureLookups`.

In [0]:
# End any existing runs (in the case this notebook is being run for a second time)
mlflow.end_run()

# Start an mlflow run, which is needed for the feature store to log the model
mlflow.start_run() 

# Since the rounded timestamp columns would likely cause the model to overfit the data 
# unless additional feature engineering was performed, exclude them to avoid training on them.
exclude_columns = ["rounded_pickup_datetime", "rounded_dropoff_datetime"]

# Create the training set that includes the raw input data merged with corresponding features from both feature tables
training_set = fs.create_training_set(
  taxi_data,
  feature_lookups = pickup_feature_lookups + dropoff_feature_lookups,
  label = "fare_amount",
  exclude_columns = exclude_columns
)

# Load the TrainingSet into a dataframe which can be passed into sklearn for training a model
training_df = training_set.load_df()

In [0]:
# Display the training dataframe, and note that it contains both the raw input data and the features from the Feature Store, like `dropoff_is_weekend`
display(training_df)

trip_distance,pickup_zip,dropoff_zip,mean_fare_window_1h_pickup_zip,count_trips_window_1h_pickup_zip,count_trips_window_30m_dropoff_zip,dropoff_is_weekend,fare_amount
5.35,10003,11238,14.333333,3,1,0,18.5
6.5,10282,10001,21.5,1,2,0,21.5
0.9,10119,10003,5.5,1,1,0,5.5
3.5,10001,11222,null,null,null,null,13.5
0.3,10028,10028,4.25,2,1,0,3.5
0.0,10038,10005,5.0,1,1,0,5.0
6.57,10001,11377,21.5,1,1,0,21.5
1.08,10103,10167,16.75,2,1,0,11.5
0.8,10003,10011,6.25,2,2,1,7.0
1.3,10199,10020,9.0,1,2,1,9.0


Train a LightGBM model on the data returned by `TrainingSet.to_df`, then log the model with `FeatureStoreClient.log_model`. The model will be packaged with feature metadata.

In [0]:
from sklearn.model_selection import train_test_split
from mlflow.tracking import MlflowClient
import lightgbm as lgb
import mlflow.lightgbm
from mlflow.models.signature import infer_signature

features_and_label = training_df.columns

# Collect data into a Pandas array for training
data = training_df.toPandas()[features_and_label]

train, test = train_test_split(data, random_state=123)
X_train = train.drop(["fare_amount"], axis=1)
X_test = test.drop(["fare_amount"], axis=1)
y_train = train.fare_amount
y_test = test.fare_amount

mlflow.lightgbm.autolog()
train_lgb_dataset = lgb.Dataset(X_train, label=y_train.values)
test_lgb_dataset = lgb.Dataset(X_test, label=y_test.values)

param = {"num_leaves": 32, "objective": "regression", "metric": "rmse"}
num_rounds = 100

# Train a lightGBM model
model = lgb.train(
  param, train_lgb_dataset, num_rounds
)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000649 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 758
[LightGBM] [Info] Number of data points in the train set: 16449, number of used features: 7
[LightGBM] [Info] Start training from score 12.304533
2021/08/18 13:53:17 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/databricks/python/lib/python3.8/site-packages/mlflow/models/signature.py:127: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."

In [0]:
# Log the trained model with MLflow and package it with feature lookup information. 
fs.log_model(
  model,
  artifact_path="model_packaged",
  flavor=mlflow.lightgbm,
  training_set=training_set,
  registered_model_name="taxi_example_fare_packaged"
)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3790025709814498> in <module> 
 1 # Log the trained model with MLflow and package it with feature lookup information. 
 2 fs.log_model(
 ----> 3 model , 
 4 artifact_path = "iris_decision_tree_model_packaged" , 
 5 flavor = mlflow . sklearn , 

 NameError : name 'model' is not defined

## Scoring: Batch Inference

Suppose another data scientist now wants to apply this model to a different batch of data.

In [0]:
new_taxi_data = rounded_taxi_data(raw_data)

Display the data to use for inference, reordered to highlight the `fare_amount` column, which is the prediction target.

In [0]:
cols = ['fare_amount', 'trip_distance', 'pickup_zip', 'dropoff_zip', 'rounded_pickup_datetime', 'rounded_dropoff_datetime']
new_taxi_data_reordered = new_taxi_data.select(cols)
display(new_taxi_data_reordered)

fare_amount,trip_distance,pickup_zip,dropoff_zip,rounded_pickup_datetime,rounded_dropoff_datetime
19.0,4.94,10282,10171,1455469200,1455471000
3.5,0.28,10110,10110,1454611500,1454612400
5.0,0.7,10103,10023,1455729300,1455730200
6.0,0.8,10022,10017,1455792300,1455793200
17.0,4.51,10110,10282,1456150500,1456153200
7.0,1.8,10009,10065,1454655600,1454655600
12.0,2.58,10153,10199,1455549300,1455550200
11.0,1.4,10112,10069,1456427700,1456428600
7.5,1.21,10023,10153,1455381000,1455382800
6.0,0.6,10012,10003,1455408900,1455409800


Use the `score_batch` API to evaluate the model on the batch of data, retrieving needed features from FeatureStore.

In [0]:
# Get the model URI
latest_model_version = get_latest_model_version("taxi_example_fare_packaged")
model_uri = f"models:/taxi_example_fare_packaged/{latest_model_version}"

# Call score_batch to get the predictions from the model
with_predictions = fs.score_batch(model_uri, new_taxi_data)

In [0]:
display(with_predictions)

trip_distance,fare_amount,pickup_zip,dropoff_zip,rounded_pickup_datetime,rounded_dropoff_datetime,mean_fare_window_1h_pickup_zip,count_trips_window_1h_pickup_zip,count_trips_window_30m_dropoff_zip,dropoff_is_weekend,prediction
5.35,18.5,10003,11238,1455662700,1455663600,14.333333,3,1,0,20.517827066609193
6.5,21.5,10282,10001,1454688900,1454689800,21.5,1,2,0,21.866937376286817
0.9,5.5,10119,10003,1454917500,1454918400,5.5,1,1,0,5.585403308600373
3.5,13.5,10001,11222,1456785000,1456786800,null,null,null,null,14.473453841847562
0.3,3.5,10028,10028,1454520600,1454520600,4.25,2,1,0,3.519921012144704
0.0,5.0,10038,10005,1455066000,1455066000,5.0,1,1,0,3.011963630171005
6.57,21.5,10001,11377,1455852600,1455854400,21.5,1,1,0,21.666255297643087
1.08,11.5,10103,10167,1454422500,1454423400,16.75,2,1,0,8.316411891362742
0.8,7.0,10003,10011,1455983100,1455984000,6.25,2,2,1,5.974961989732555
1.3,9.0,10199,10020,1455467400,1455469200,9.0,1,2,1,8.827062145789471


<img src="https://docs.databricks.com/_static/images/machine-learning/feature-store/taxi_example_score_batch.png"/>

### View the taxi fare predictions

This code reorders the columns to show the taxi fare predictions in the first column.  Note that the `predicted_fare_amount` roughly lines up with the actual `fare_amount`, although more data and feature engineering would be required to improve the model accuracy.

In [0]:
import pyspark.sql.functions as func

cols = ['prediction', 'fare_amount', 'trip_distance', 'pickup_zip', 'dropoff_zip', 
        'rounded_pickup_datetime', 'rounded_dropoff_datetime', 'mean_fare_window_1h_pickup_zip', 
        'count_trips_window_1h_pickup_zip', 'count_trips_window_30m_dropoff_zip', 'dropoff_is_weekend']

with_predictions_reordered = (
    with_predictions.select(
        cols,
    )
    .withColumnRenamed(
        "prediction",
        "predicted_fare_amount",
    )
    .withColumn(
      "predicted_fare_amount",
      func.round("predicted_fare_amount", 2),
    )
)

display(with_predictions_reordered)

predicted_fare_amount,fare_amount,trip_distance,pickup_zip,dropoff_zip,rounded_pickup_datetime,rounded_dropoff_datetime,mean_fare_window_1h_pickup_zip,count_trips_window_1h_pickup_zip,count_trips_window_30m_dropoff_zip,dropoff_is_weekend
20.52,18.5,5.35,10003,11238,1455662700,1455663600,14.333333,3,1,0
21.87,21.5,6.5,10282,10001,1454688900,1454689800,21.5,1,2,0
5.59,5.5,0.9,10119,10003,1454917500,1454918400,5.5,1,1,0
14.47,13.5,3.5,10001,11222,1456785000,1456786800,null,null,null,null
3.52,3.5,0.3,10028,10028,1454520600,1454520600,4.25,2,1,0
3.01,5.0,0.0,10038,10005,1455066000,1455066000,5.0,1,1,0
21.67,21.5,6.57,10001,11377,1455852600,1455854400,21.5,1,1,0
8.32,11.5,1.08,10103,10167,1454422500,1454423400,16.75,2,1,0
5.97,7.0,0.8,10003,10011,1455983100,1455984000,6.25,2,2,1
8.83,9.0,1.3,10199,10020,1455467400,1455469200,9.0,1,2,1


## Next steps

1. Explore the feature tables created in this example in the <a href="#feature-store">Feature Store UI</a>.
1. Adapt this notebook to your own data and create your own feature tables.